## Objective
Eats4Life would like to update its menu to include wine suggestions with each of its main entrees (defined by the meat selection). The owner would like to take a Data Analytics approach and explore data he collected over the past several years on main courses (meat) and wine that was ordered with it. Eats4Life is open to listing more than one wine for each main entree, but only if the data supports it. The scope of services requested includes:

- Summary information on the main entrees (meat)
- Wine suggestion(s) for **each** main entree along with supporting information as to why this (these) wines are suggested for the entrée (if you have no suggested wine for a given entrée, provide information as to why this is your suggestion)
- Any other information of interest in terms of customer order habits

## Data Provided
The dataset `orderData.csv` has three columns:

- `orderNo` – identifies each table/party that sat at the restaurant
- `seatNo` – indicates which seat at the table ordered each meal
- `item` – the item that was ordered

The data has been cleaned, so that each order contains three items per individual: a meat, a side, and a wine.

In [10]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

In [11]:
df = pd.read_csv('https://raw.githubusercontent.com/sjsimmo2/DataMining-Fall/refs/heads/master/orderData.csv')

In [12]:
print(df.shape)
df.head()

(228699, 3)


,orderNo,seatNo,item
0,122314,1,Salmon
1,122314,1,Oyster Bay Sauvignon Blanc
2,122314,1,Bean Trio
3,122314,2,Pork Chop
4,122314,2,Three Rivers Red


In [23]:
df['item'].value_counts()

item
Seasonal Veg                          14574
Filet Mignon                          13407
Sea Bass                              12302
Duckhorn Chardonnay                   11723
Bean Trio                             11696
Roasted Root Veg                      11323
Pork Tenderloin                       11138
Pork Chop                             10976
Warm Goat Salad                       10605
Adelsheim Pinot Noir                  10308
Roasted Potatoes                       9847
Salmon                                 9336
Caesar Salad                           9168
Mashed Potatoes                        9020
Blackstone Merlot                      8485
Total Recall Chardonnay                8012
Duck Breast                            7915
Single Vineyard Malbec                 7791
Swordfish                              7439
Innocent Bystander Sauvignon Blanc     6397
Oyster Bay Sauvignon Blanc             4815
Echeverria Gran Syrah                  4600
Brancott Pinot Grigio      

In [24]:
meats = [
    "Filet Mignon",
    "Sea Bass",
    "Pork Tenderloin",
    "Pork Chop",
    "Salmon",
    "Duck Breast",
    "Swordfish",
    "Roast Chicken"
]

sides = [
    "Seasonal Veg",
    "Bean Trio",
    "Roasted Root Veg",
    "Warm Goat Salad",
    "Roasted Potatoes",
    "Caesar Salad",
    "Mashed Potatoes"
]

wines = [
    "Duckhorn Chardonnay",
    "Adelsheim Pinot Noir",
    "Blackstone Merlot",
    "Total Recall Chardonnay",
    "Single Vineyard Malbec",
    "Innocent Bystander Sauvignon Blanc",
    "Oyster Bay Sauvignon Blanc",
    "Echeverria Gran Syrah",
    "Brancott Pinot Grigio",
    "Cantina Pinot Bianco",
    "Louis Rouge",
    "Helben Blanc",
    "Three Rivers Red"
]

## Data Processing

In [25]:
#create a df with only the meat and wine pairings
df_meat_wine = df[df['item'].isin(meats + wines)]

#create a dummy varriable for each item
df_1 = pd.get_dummies(df_meat_wine["item"])*1

#add the original order number to the new df
df_1["orderNo"] = df_meat_wine["orderNo"]
#add the original seat number to the new df
df_1['seatNo'] = df_meat_wine['seatNo']
#df_1["order_seatNo"] = df['orderNo'].astype(str) + "_" + df['seatNo'].astype(str)

#group by orderNo and seatNo, then calculates the maximum value for each col
df_1 = df_1.groupby(['orderNo', 'seatNo']).max()

#convert the dummy vars back to boolean 
preprocessed_df = df_1.map(bool)

preprocessed_df.head()

Adelsheim Pinot Noir  Blackstone Merlot  \
orderNo seatNo                                            
122314  1                      False              False   
        2                      False              False   
        3                      False              False   
        4                      False              False   
        5                      False              False   

                Brancott Pinot Grigio  Cantina Pinot Bianco  Duck Breast  \
orderNo seatNo                                                             
122314  1                       False                 False        False   
        2                       False                 False        False   
        3                       False                 False        False   
        4                       False                 False        False   
        5                       False                 False         True   

                Duckhorn Chardonnay  Echeverria Gran Syrah  Filet Mignon  \
orderNo seatNo                                                             
122314  1                     False                  False         False   
        2                     False                  False         False   
        3                     False                  False         False   
        4                     False                  False         False   
        5                     False                  False         False   

                Helben Blanc  Innocent Bystander Sauvignon Blanc  ...  \
orderNo seatNo                                                    ...   
122314  1              False                               False  ...   
        2              False                               False  ...   
        3              False                               False  ...   
        4              False                               False  ...   
        5              False                                True  ...   

                Oyster Bay Sauvignon Blanc  Pork Chop  Pork Tenderloin  \
orderNo seatNo                                                           
122314  1                             True      False            False   
        2                            False       True            False   
        3                             True      False            False   
        4                            False      False            False   
        5                            False      False            False   

                Roast Chicken  Salmon  Sea Bass  Single Vineyard Malbec  \
orderNo seatNo                                                            
122314  1               False    True     False                   False   
        2               False   False     False                   False   
        3               False   False      True                   False   
        4               False   False      True                   False   
        5               False   False     False                   False   

                Swordfish  Three Rivers Red  Total Recall Chardonnay  
orderNo seatNo                                                        
122314  1           False             False                    False  
        2           False              True                    False  
        3           False             False                    False  
        4           False             False                     True  
        5           False             False                    False  

[5 rows x 21 columns]

## apriori algorithm

In [27]:
#apriori algorithm
food_wine_assoc = apriori(preprocessed_df, min_support = 0.001, use_colnames = True)

#association rules
out_rules = association_rules(food_wine_assoc,metric = "confidence", min_threshold = 0.1)

#select columns from the output
out_rules2=out_rules[['antecedents','consequents','support','confidence','lift']]

out_rules2

,antecedents,consequents,support,confidence,lift
0,(Adelsheim Pinot Noir),(Duck Breast),0.016476,0.121847,1.173565
1,(Duck Breast),(Adelsheim Pinot Noir),0.016476,0.158686,1.173565
2,(Filet Mignon),(Adelsheim Pinot Noir),0.049650,0.282315,2.087867
3,(Adelsheim Pinot Noir),(Filet Mignon),0.049650,0.367191,2.087867
4,(Adelsheim Pinot Noir),(Pork Chop),0.013826,0.102251,0.710175
...,...,...,...,...,...
80,(Three Rivers Red),(Sea Bass),0.004854,0.260931,1.616936
81,(Total Recall Chardonnay),(Sea Bass),0.015990,0.152147,0.942823
82,(Three Rivers Red),(Swordfish),0.001928,0.103667,1.062355
83,(Swordfish),(Total Recall Chardonnay),0.017447,0.178787,1.701136


In [29]:
meats_antecedents = out_rules2[out_rules2['antecedents'].apply(lambda x: list(x)[0] in meats)]
meats_antecedents


,antecedents,consequents,support,confidence,lift
1,(Duck Breast),(Adelsheim Pinot Noir),0.016476,0.158686,1.173565
2,(Filet Mignon),(Adelsheim Pinot Noir),0.049650,0.282315,2.087867
6,(Pork Tenderloin),(Adelsheim Pinot Noir),0.043734,0.299336,2.213742
7,(Duck Breast),(Blackstone Merlot),0.032886,0.316740,2.845736
9,(Filet Mignon),(Blackstone Merlot),0.062729,0.356679,3.204565
13,(Roast Chicken),(Brancott Pinot Grigio),0.008448,0.173118,2.947147
15,(Sea Bass),(Brancott Pinot Grigio),0.016856,0.104455,1.778223
17,(Swordfish),(Brancott Pinot Grigio),0.014705,0.150692,2.565370
22,(Salmon),(Cantina Pinot Bianco),0.013328,0.108826,2.308330
30,(Duck Breast),(Single Vineyard Malbec),0.010533,0.101453,0.992692
